<a href="https://colab.research.google.com/github/zjuer-dm/colab/blob/main/2d_gaussian_splatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before proceeding, please ensure that your notebook is connected to a GPU runtime.

In [ ]:
%cd /content
!git clone https://github.com/atakan-topaloglu/2d-gaussian-splatting --recursive
!pip install -q plyfile

/content
Cloning into '2d-gaussian-splatting'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 238 (delta 110), reused 77 (delta 77), pack-reused 66 (from 1)
Receiving objects: 100% (238/238), 9.39 MiB | 20.91 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Submodule 'submodules/diff-surfel-rasterization' (https://github.com/hbb1/diff-surfel-rasterization.git) registered for path 'submodules/diff-surfel-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/2d-gaussian-splatting/submodules/diff-surfel-rasterization'...
remote: Enumerating objects: 697, done.        
remote: Counting objects: 100% (287/287), done.        
remote: Compressing objects: 100% (33/33), done.        
remote: Total 697 (delta 272), reused 254 (delta 254), pack-reused 410 (from 1)        
Receivi

In [ ]:
%cd /content/2d-gaussian-splatting/

In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda' #ensure that the CUDA path is correct
!pip install ninja
!pip install submodules/simple-knn
!pip install submodules/diff-surfel-rasterization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/2d-gaussian-splatting/

For Colab to access your COLMAP processed data, please copy the COLMAP output to your Google Drive and enter the full COLMAP source path (e.g. /content/drive/MyDrive/COLMAP_out)

In [ ]:
COLMAP_PATH = str(input('Please enter COLMAP output path:'))

In [ ]:
!python train.py -s $COLMAP_PATH

Training is complete, to use render.py, please enter the desired model_path located in your Drive: /content/drive/MyDrive/2d_gaussian_splatting/output/{10 digit UUID}: (e.g: /content/drive/MyDrive/2d_gaussian_splatting/output/8d812792-1)

In [ ]:
MODEL_PATH = str(input('Please enter the model path:'))

In [ ]:
!pip3 install open3d==0.18.0 mediapy==1.1.2 lpips==0.1.4 scikit-image==0.21.0 tqdm==4.66.2 trimesh==4.3.2 opencv-python

In [ ]:
!python render.py -m $MODEL_PATH -s $COLMAP_PATH

Training and rendering is complete, please navigate to your drive and see the model outputs under the directory "2d_gaussian_splatting/".